# جلسه ۵ – هماهنگ‌کننده چند‌عاملی

نمایش یک خط لوله ساده دو‌عاملی (پژوهشگر -> ویراستار) با استفاده از Foundry Local.


### توضیح: نصب وابستگی‌ها
`foundry-local-sdk` و `openai` را نصب می‌کند که برای دسترسی به مدل محلی و تکمیل چت مورد نیاز هستند. این عملیات بی‌اثر است.


# سناریو
الگوی هماهنگ‌کننده دو عامل حداقلی را پیاده‌سازی می‌کند:
- **عامل پژوهشگر** نکات واقعی مختصر را جمع‌آوری می‌کند
- **عامل ویراستار** برای وضوح اجرایی بازنویسی می‌کند

نشان‌دهنده حافظه مشترک برای هر عامل، انتقال متوالی خروجی میانی و یک تابع خط لوله ساده است. قابل گسترش به نقش‌های بیشتر (مانند منتقد، تأییدکننده) یا شاخه‌های موازی.

**متغیرهای محیطی:**
- `FOUNDRY_LOCAL_ALIAS` - مدل پیش‌فرض برای استفاده (پیش‌فرض: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - مدل عامل اصلی (جایگزین ALIAS)
- `AGENT_MODEL_EDITOR` - مدل عامل ویراستار (پیش‌فرض: اصلی)

**مرجع SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**نحوه کار:**
1. **FoundryLocalManager** به طور خودکار سرویس Foundry Local را راه‌اندازی می‌کند
2. مدل مشخص‌شده را دانلود و بارگذاری می‌کند (یا از نسخه ذخیره‌شده استفاده می‌کند)
3. یک نقطه پایانی سازگار با OpenAI برای تعامل فراهم می‌کند
4. هر عامل می‌تواند از مدل متفاوتی برای وظایف تخصصی استفاده کند
5. منطق بازگشت داخلی به طور مؤثر خرابی‌های گذرا را مدیریت می‌کند

**ویژگی‌های کلیدی:**
- ✅ کشف و راه‌اندازی خودکار سرویس
- ✅ مدیریت چرخه عمر مدل (دانلود، ذخیره، بارگذاری)
- ✅ سازگاری با SDK OpenAI برای API آشنا
- ✅ پشتیبانی از چند مدل برای تخصص عامل
- ✅ مدیریت خطاهای قوی با منطق بازگشت
- ✅ استنتاج محلی (بدون نیاز به API ابری)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### توضیح: وارد کردن‌های اصلی و تایپینگ  
معرفی دیتاکلاس‌ها برای ذخیره‌سازی پیام‌های عامل و استفاده از تایپینگ برای وضوح بیشتر. وارد کردن مدیر محلی Foundry و کلاینت OpenAI برای اقدامات بعدی عامل.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### توضیح: مقداردهی اولیه مدل (الگوی SDK)
از SDK محلی پایتون Foundry برای مدیریت قدرتمند مدل استفاده می‌شود:
- **FoundryLocalManager(alias)** - سرویس را به‌صورت خودکار راه‌اندازی کرده و مدل را با استفاده از alias بارگذاری می‌کند
- **get_model_info(alias)** - alias را به شناسه مدل مشخص تبدیل می‌کند
- **manager.endpoint** - نقطه پایانی سرویس را برای کلاینت OpenAI فراهم می‌کند
- **manager.api_key** - کلید API را فراهم می‌کند (اختیاری برای استفاده محلی)
- پشتیبانی از مدل‌های جداگانه برای عوامل مختلف (اصلی در مقابل ویرایشگر)
- منطق داخلی تلاش مجدد با بازگشت نمایی برای افزایش مقاومت
- تأیید اتصال برای اطمینان از آماده بودن سرویس

**الگوی کلیدی SDK:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**مدیریت چرخه عمر:**
- مدیران به‌صورت جهانی ذخیره می‌شوند تا پاک‌سازی مناسب انجام شود
- هر عامل می‌تواند از مدل متفاوتی برای تخصصی‌سازی استفاده کند
- کشف خودکار سرویس و مدیریت اتصال
- تلاش مجدد با بازگشت نمایی به‌صورت روان در صورت بروز خطا

این موارد اطمینان حاصل می‌کنند که مقداردهی اولیه به‌درستی قبل از شروع هماهنگی عوامل انجام شود.

**مرجع:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### توضیح: کلاس‌های Agent و Memory
تعریف `AgentMsg` سبک برای ورودی‌های حافظه و `Agent` که شامل موارد زیر است:
- **نقش سیستم** - شخصیت و دستورالعمل‌های عامل
- **تاریخچه پیام‌ها** - حفظ زمینه مکالمه
- **متد act()** - اجرای اقدامات با مدیریت صحیح خطاها

عامل می‌تواند از مدل‌های مختلف (اصلی در مقابل ویرایشگر) استفاده کند و زمینه‌ای جداگانه برای هر عامل حفظ کند. این الگو امکان موارد زیر را فراهم می‌کند:
- حفظ حافظه در طول اقدامات
- تخصیص انعطاف‌پذیر مدل برای هر عامل
- جداسازی خطاها و بازیابی
- زنجیره‌سازی و هماهنگی آسان


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### توضیح: خط لوله هماهنگ‌شده
دو عامل تخصصی ایجاد می‌کند:
- **پژوهشگر**: از مدل اصلی استفاده می‌کند و اطلاعات واقعی جمع‌آوری می‌کند
- **ویرایشگر**: می‌تواند از مدل جداگانه (در صورت پیکربندی) استفاده کند، اطلاعات را اصلاح و بازنویسی می‌کند

تابع `pipeline`:
1. پژوهشگر اطلاعات خام را جمع‌آوری می‌کند
2. ویرایشگر آن را به خروجی آماده برای اجرا تبدیل می‌کند
3. هر دو نتیجه میانی و نهایی را بازمی‌گرداند

این الگو امکان‌پذیر می‌کند:
- تخصص مدل (مدل‌های مختلف برای نقش‌های مختلف)
- بهبود کیفیت از طریق پردازش چندمرحله‌ای
- قابلیت ردیابی تحول اطلاعات
- گسترش آسان به عوامل بیشتر یا پردازش موازی


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### توضیح: اجرای خط لوله و نتایج
اجرای خط لوله چندعاملی بر روی یک سؤال با موضوع انطباق و تأخیر برای نشان دادن:
- تبدیل اطلاعات چندمرحله‌ای
- تخصص و همکاری عوامل
- بهبود کیفیت خروجی از طریق اصلاح
- قابلیت ردیابی (حفظ خروجی‌های میانی و نهایی)

**ساختار نتیجه:**
- `question` - پرسش اصلی کاربر
- `research` - خروجی تحقیق خام (نکات واقعی)
- `final` - خلاصه اجرایی اصلاح‌شده
- `models` - مدل‌هایی که در هر مرحله استفاده شده‌اند

**ایده‌های توسعه:**
1. افزودن عامل منتقد برای بررسی کیفیت
2. اجرای عوامل تحقیق موازی برای جنبه‌های مختلف
3. افزودن عامل تأییدکننده برای بررسی صحت اطلاعات
4. استفاده از مدل‌های مختلف برای سطوح پیچیدگی متفاوت
5. پیاده‌سازی حلقه‌های بازخورد برای بهبود تدریجی


### پیشرفته: پیکربندی سفارشی عامل

سعی کنید رفتار عامل را با تغییر متغیرهای محیطی قبل از اجرای سلول اولیه‌سازی سفارشی کنید:

**مدل‌های موجود:**
- از دستور `foundry model ls` در ترمینال استفاده کنید تا تمام مدل‌های موجود را ببینید
- مثال‌ها: phi-4-mini، phi-3.5-mini، qwen2.5-7b، llama-3.2-3b و غیره.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم ترجمه‌ها دقیق باشند، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما هیچ مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
